# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch
import numpy as np

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""

# TODO Implement 

twits_num = len(twits['data'])
print(twits_num)

1548010


### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [5]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    text = re.sub(r'https?://\S+',' ',text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub(r'\$[A-z]+',' ',text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r'\@\S+',' ',text)

    # Replace everything not a letter with a space
    text = re.sub(r'[^a-zA-Z]+',' ', text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split()

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(w,pos='v') for w in tokens if len(w)>1]
    
    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [6]:
# TODO Implement
tokenized = [preprocess(message) for message in messages]

In [7]:
from collections import Counter


"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 
words = []
for tokens in tokenized:
    for word in tokens:
        words.append(word)
        
bow = Counter(words)

### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [8]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
freq = {word:count/sum(bow.values()) for word,count in bow.items()}

freq_s = sorted(freq, key = freq.get, reverse = True)
# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 7.1e-8

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 90000

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [word for word,count in bow.items() if count > high_cutoff ]


filtered_words = [word for word in freq_s if (freq[word] > low_cutoff and word not in K_most_common)]
print(K_most_common)


['buy', 'at', 'for', 'of', 'be', 'on', 'the', 'have', 'to', 'in', 'short', 'and', 'this', 'it', 'go', 'you', 'will', 'up', 'report']


In [9]:
print('unique words after filtering:{}'.format(len(filtered_words))) 
print('all text unique words:{}'.format(len(freq)))                             

unique words after filtering:45315
all text unique words:95492


### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [10]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {word:ii for ii,word in enumerate(filtered_words,1)}
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {ii:word for word,ii in vocab.items()}


In [18]:
# tokenized with the words not in `filtered_words` removed.
filtered = []
for tokens in tokenized:
    filtered.append([word for word in tokens if word in filtered_words])

In [ ]:
len(filtered)

In [ ]:
sample_filter = filtered[900:1000]
len(sample_filter)

In [149]:
f = open('filtered_all.txt','w')
for review in sample_filter:
    s = ' '.join(review)
    f.write(s)
    f.write('\n')
    f.close

In [11]:
filter_all = []
import os
files = open('filtered_all.txt','r')
contents = files.readlines()
for msg in contents:
    msg = msg.strip('\n')
    list_1 = msg.split()
    filter_all.append(list_1)
files.close()

In [12]:
len(filter_all)

1548010

### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [13]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filter_all[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [14]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.19512780942990196

Finally let's convert our tokens into integer ids which we can pass to the network.

In [15]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

In [16]:
print('the length of token_ids: {}'.format(len(token_ids)))
print('the length of sentiments: {}'.format(len(sentiments)))

the length of token_ids: 1028367
the length of sentiments: 1028367


In [17]:
token_ids[:10]

[[141, 1509, 127],
 [519, 1349, 910, 12, 137, 48, 246, 152, 47, 656, 202, 223, 547],
 [538, 85, 185, 138, 103, 429, 138, 66, 1310, 103, 382, 1085],
 [114, 102, 1387, 1168, 72, 20, 367, 143],
 [63, 297, 215, 23, 577],
 [482, 40, 14, 197, 13, 102, 47, 65, 570, 70],
 [177, 162, 14, 101, 4, 41, 136],
 [102, 155, 55, 60, 728, 18, 44, 124, 311, 41, 18, 1901, 118, 3755, 2, 2439],
 [147],
 [308, 460, 99]]

In [18]:
sentiments[:10]

[4, 4, 3, 3, 0, 3, 4, 0, 4, 0]

In [63]:
import numpy as np
a = np.array([[[2,3,10,1],[1,4,5,3],[2,7,9,11]],[[5,8,6,2],[7,9,0,8],[3,5,7,6]]])
print(a.shape)
a

(2, 3, 4)


array([[[ 2,  3, 10,  1],
        [ 1,  4,  5,  3],
        [ 2,  7,  9, 11]],

       [[ 5,  8,  6,  2],
        [ 7,  9,  0,  8],
        [ 3,  5,  7,  6]]])

In [101]:
b = a[-1,:,:]
print(b)
b.shape

[[5 8 6 2]
 [7 9 0 8]
 [3 5 7 6]]


(3, 4)

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [44]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement

        # Setup embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_size)
               
        # Setup additional layers
        self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers, dropout=dropout, batch_first=False)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(lstm_size,output_size)
        self.softmax = nn.LogSoftmax(dim=1)


    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                  weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        
        return hidden


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        # TODO Implement 
        batch_size = nn_input.size(1)
                  
        embeds = self.embedding(nn_input)
        lstm_out, h = self.lstm(embeds, hidden_state)
                  
        lstm_out = lstm_out.contiguous().view(-1, self.lstm_size)
                  
        out = self.dropout(lstm_out)
        out = self.fc(out)
                  
        out_softmax = self.softmax(out)
        output = out_softmax.view(-1, batch_size, self.output_size)
        output = output[-1,:,:]
                  
        return output, h

### View Model

In [45]:

model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)

In [46]:
logps, _ = model.forward(input, hidden)
print(logps)
logps.shape

tensor([[-1.4014, -1.7752, -1.7685, -1.6408, -1.5146],
        [-1.4769, -1.7189, -1.7653, -1.5636, -1.5518],
        [-1.5628, -1.6299, -1.8260, -1.5350, -1.5233],
        [-1.3947, -1.7885, -1.7733, -1.6268, -1.5208]])


torch.Size([4, 5])

## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [26]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [27]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# TODO Implement 
split_size = 0.8
split_idx = int(len(token_ids)*0.8)
print(split_idx)

train_features = token_ids[:split_idx]
val_features = token_ids[split_idx:]
train_labels = sentiments[:split_idx]
val_labels = sentiments[split_idx:]

822693


In [28]:
print('the shape of train_features: {}'.format(len(train_features)))
print('the shape of valid_features: {}'.format(len(val_features)))
print('the shape of train_labels: {}'.format(len(train_labels)))
print('the shape of valid_labels: {}'.format(len(val_labels)))

the shape of train_features: 822693
the shape of valid_features: 205674
the shape of train_labels: 822693
the shape of valid_labels: 205674


In [29]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length = 20, batch_size = 64)))
print(text_batch.shape)
print(labels.shape)
print(text_batch)

torch.Size([20, 64])
torch.Size([64])
tensor([[     0,      0,      0,  ...,      0,      0,      0],
        [     0,      0,      0,  ...,      0,      0,      0],
        [     0,      0,      0,  ...,      0,      0,      0],
        ...,
        [   141,    202,    103,  ...,     53,    307,     41],
        [  1509,    223,    382,  ...,  10248,     31,     25],
        [   127,    547,   1085,  ...,   7496,     40,    635]])


In [30]:
text_batch, labels = next(iter(dataloader(val_features, val_labels, sequence_length = 20, batch_size = 64)))
print(text_batch.shape)
print(labels.shape)
print(labels)

torch.Size([20, 64])
torch.Size([64])
tensor([ 3,  0,  3,  3,  0,  0,  4,  3,  0,  0,  1,  3,  0,  1,
         1,  2,  3,  2,  2,  1,  2,  4,  0,  0,  2,  4,  3,  4,
         3,  2,  2,  1,  1,  4,  4,  0,  2,  1,  2,  1,  1,  2,
         0,  4,  3,  1,  3,  3,  1,  3,  2,  3,  2,  0,  3,  1,
         4,  0,  1,  1,  3,  4,  4,  1])


In [52]:
for text_batch,labels in dataloader(train_features[:-37], train_labels[:-37], sequence_length=20, batch_size=128, shuffle=True):
    print(labels.shape)   

torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size

In [54]:
for val_batch,val_label in dataloader(val_features[:-106], val_labels[:-106], sequence_length=20, batch_size=128, shuffle=True):
    print(val_label.shape)   

torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size

In [35]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

### Training
It's time to train the neural network!

In [59]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.3)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(45316, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.3)
  (dropout): Dropout(p=0.3)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (softmax): LogSoftmax()
)

In [60]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs = 3
batch_size = 128
learning_rate = 0.003
clip = 5

print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    
    steps = 0
    h = model.init_hidden(batch_size)
    
    for text_batch,labels in dataloader(train_features[:-37], train_labels[:-37], sequence_length=20, 
                                        batch_size=batch_size, shuffle=True):
        
        steps += 1
         
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        
        h = tuple([each.data.to(device) for each in h])
        
        
        
        # TODO Implement: Train Model
        model.zero_grad()
        
        output, h = model(text_batch, h)
        
        loss = criterion(output, labels)
        
        loss.backward()
        
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if steps % print_every == 0:
            
            model.eval()
            
            # TODO Implement: Print metrics
            
            val_hidden = model.init_hidden(batch_size)
            val_losses = []
            accuracy = []
            for val_batch, val_label in dataloader(val_features[:-106], val_labels[:-106], sequence_length=20, batch_size=batch_size, shuffle=True):
                
                val_batch, val_label = val_batch.to(device), val_label.to(device)              
                
                val_h = tuple([each.data.to(device) for each in val_hidden])
                                
                val_out, val_h = model(val_batch, val_h)
                val_loss = criterion(val_out, val_label)
                val_losses.append(val_loss.item())
                ps = torch.exp(val_out)
                top_p,top_class = ps.topk(1,dim=1)
                equals = top_class == val_label.view(*top_class.shape)
                acc = torch.mean(equals.type(torch.FloatTensor))
                accuracy.append(acc)
                
            model.train()    
            print('Step: {}...'.format(steps), 
                  'Training Loss: {:.4f}'.format(loss.item()), 
                 'Valid Loss: {:.4f}'.format(np.mean(val_losses)),
                 'Accuacy: {:.4f}'.format(np.mean(accuracy)))
            

Starting epoch 1
Step: 100... Training Loss: 1.2340 Valid Loss: 1.1891 Accuacy: 0.5064
Step: 200... Training Loss: 1.1151 Valid Loss: 1.0930 Accuacy: 0.5565
Step: 300... Training Loss: 0.9707 Valid Loss: 1.0584 Accuacy: 0.5749
Step: 400... Training Loss: 0.9336 Valid Loss: 1.0274 Accuacy: 0.5890
Step: 500... Training Loss: 0.9578 Valid Loss: 1.0118 Accuacy: 0.5952
Step: 600... Training Loss: 0.9615 Valid Loss: 1.0226 Accuacy: 0.5814
Step: 700... Training Loss: 1.0977 Valid Loss: 0.9829 Accuacy: 0.6111
Step: 800... Training Loss: 0.9748 Valid Loss: 0.9766 Accuacy: 0.6156
Step: 900... Training Loss: 0.9359 Valid Loss: 0.9738 Accuacy: 0.6097
Step: 1000... Training Loss: 1.0234 Valid Loss: 0.9714 Accuacy: 0.6146
Step: 1100... Training Loss: 0.8521 Valid Loss: 0.9586 Accuacy: 0.6232
Step: 1200... Training Loss: 0.9157 Valid Loss: 0.9554 Accuacy: 0.6195
Step: 1300... Training Loss: 0.8366 Valid Loss: 0.9505 Accuacy: 0.6284
Step: 1400... Training Loss: 0.9128 Valid Loss: 0.9526 Accuacy: 0.624

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [61]:
def customize_seq(batch_messages, sequence_length=20):
    batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
    for batch_num, tokens in enumerate(batch_messages):
        token_tensor = torch.tensor(tokens)
        start_idx = max(sequence_length - len(token_tensor), 0)
        batch[start_idx:, batch_num] = token_tensor[:sequence_length]
    return batch

In [70]:
a = "Google is working on self driving cars, I'm bullish on $goog"
tokens = preprocess(a)
print(tokens)  
token2 = [word for word in tokens if word in filtered_words]

print(token2)
token3 = [vocab[word] for word in token2]
print(token3)

['google', 'be', 'work', 'on', 'self', 'drive', 'cars', 'bullish', 'on']
['google', 'work', 'self', 'drive', 'cars', 'bullish']
[619, 284, 1598, 668, 1492, 89]


In [91]:
def customerize_seq(text_token, sequence_length):
    
     #sequence_length = 20
    batch = torch.zeros((20, 1), dtype=torch.int64)

    token_tensor = torch.tensor(text_token)

    start_idx = max(sequence_length - len(token_tensor), 0)
    batch[start_idx:,0] = token_tensor[:sequence_length]
    return batch

In [97]:
a = customerize_seq(token3, 20)
a.size(1)

1

In [110]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [word for word in tokens if word in filtered_words]
    # Convert words to ids
    tokens = [vocab[word] for word in tokens]
    features = customerize_seq(tokens, 20)
        
    # Adding a batch dimension
    #token_tensor = torch.from_numpy(features)
    
    batch_size = features.size(1)
    # Get the NN output
    hidden = model.init_hidden(batch_size)
    logps, _ = model(features, hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    ps = torch.exp(logps)
    top_p,pred = ps.topk(1,dim=1)
    #pred = torch.round(logps.squeeze())
    
    return pred, top_p

In [111]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
predict(text, model, vocab)

(tensor([[ 3]]), tensor([[ 0.8475]]))

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** TODO: Answer Question**
The prediction of the model is 3, meaning the sentiments is positive. The probability that the sentiments are 3 is 84.75%.

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [112]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [113]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [114]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [115]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

{'symbol': '$AAPL',
 'score': (tensor([[ 4]]), tensor([[ 0.6535]])),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.